In [18]:
import pandas as pd

# Data Processing

In [19]:
folder = "/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/raw/"
intake = pd.read_excel(folder + "intake.xlsx")
outcome = pd.read_excel(folder + "outcome.xlsx")

In [20]:
intake.columns

Index(['Intake Date', 'Animal ID', 'Animal Name', 'Species', 'Primary Breed',
       'Secondary Breed', 'Sex', 'Altered', 'Intake Type', 'Intake Sub-type',
       'Intake By', 'Intake From (ID)', 'Intake From Name'],
      dtype='object')

In [21]:
intake = intake.rename(columns={"Altered":"Altered-Intake", "Animal Name":"Name"})

In [22]:
outcome.columns

Index(['Outcome Date', 'Outcome Time', 'Animal ID', 'Name', 'Species',
       'Primary Breed', 'Secondary Breed', 'Sex', 'Altered', 'Intake Type',
       'Outcome Type', 'Outcome Sub-type', 'Outcome To (ID)',
       'Outcome To Name', 'Outcome By'],
      dtype='object')

In [23]:
outcome = outcome.rename(columns={"Altered":"Altered-Outcome"})

Lots of shared columns, so I was easily able to merge intake, outcome, and length of stay data.

In [24]:
cat_data = intake.merge(outcome, on=["Animal ID", "Name", 'Species',
       'Primary Breed', 'Secondary Breed', 'Sex', "Intake Type"])

In [25]:
lengthofstay = pd.read_excel(folder + "lengthofstay.xlsx")

In [26]:
lengthofstay.columns

Index(['Animal ID', 'Animal Name', 'Species', 'Primary Breed',
       'Secondary Breed', 'Date of Birth', 'Age Group at Intake',
       'Current Status', 'Current Location', 'Intake Date', 'Intake Type',
       'Outcome Date', 'Outcome Type', 'Days in Custody', 'Days Onsite',
       'Days Available'],
      dtype='object')

I wanted to add columns for additional types of days. Cats in custody but not onsite are in foster. Cats onsite but not available are typically getting medical care or are in quarantine.

In [27]:
lengthofstay = lengthofstay.rename(columns={"Animal Name": "Name"})
lengthofstay["Days in Foster"] = (lengthofstay["Days in Custody"] 
                                  - lengthofstay["Days Onsite"])
lengthofstay["Days Medical Care"] = (lengthofstay["Days Onsite"] 
                                     - lengthofstay["Days Available"])

In [28]:
lengthofstay["Age Group at Intake"].unique()

array(['Kitten (0 weeks-1 years)', 'Adult Cat (1-10 years)',
       'Senior Cat (10+ years)', 'Adult Dog (5+ months)'], dtype=object)

In [29]:
def age_category(elem):
    # Shortening the age categories
    # there is one dog in this data, from a community euthanasia
    # but that dog is not in the intake/outcome data
    # and could be ignored
    if elem == 'Kitten (0 weeks-1 years)':
        return "Kitten"
    if elem == "Adult Cat (1-10 years)":
        return "Adult"
    if elem == "Senior Cat (10+ years)":
        return "Senior"

lengthofstay["Age Group at Intake"] = lengthofstay["Age Group at Intake"].apply(age_category)

In [30]:
cat_data = cat_data.merge(lengthofstay, on=["Animal ID", "Name", 
                                            "Species", 'Primary Breed',
                                            'Secondary Breed', 'Intake Date', 
                                            'Intake Type', 'Outcome Date', 
                                            'Outcome Type'])

In [31]:
cat_data.columns

Index(['Intake Date', 'Animal ID', 'Name', 'Species', 'Primary Breed',
       'Secondary Breed', 'Sex', 'Altered-Intake', 'Intake Type',
       'Intake Sub-type', 'Intake By', 'Intake From (ID)', 'Intake From Name',
       'Outcome Date', 'Outcome Time', 'Altered-Outcome', 'Outcome Type',
       'Outcome Sub-type', 'Outcome To (ID)', 'Outcome To Name', 'Outcome By',
       'Date of Birth', 'Age Group at Intake', 'Current Status',
       'Current Location', 'Days in Custody', 'Days Onsite', 'Days Available',
       'Days in Foster', 'Days Medical Care'],
      dtype='object')

In [32]:
# unnecessary columns: all animals are cats, and there are no "secondary breeds"
cat_data = cat_data.drop(columns=["Species", "Secondary Breed"])

In [33]:
hardtoplace = pd.read_excel(folder + "hardtoplace.xlsx")

In [34]:
hardtoplace.columns

Index(['Animal ID', 'Name', 'Attributes', 'Outcome Date', 'Outcome Type'], dtype='object')

In [35]:
# is a string-list rather than a list-list
hardtoplace["Attributes"] = hardtoplace["Attributes"].str.split(", ")

In [36]:
# separating out issues individually
hardtoplace = hardtoplace.explode("Attributes")

In [37]:
# every cat has this one
hardtoplace = hardtoplace[(hardtoplace["Attributes"] 
                           != "Standard Adoption Disclaimer")].copy()

In [38]:
def attribute_category(elem):
    # I pulled the list of unique attributes and split with domain knowledge
    medical = ['Diabetic', 'Heart murmur', 'Polydactyl', 'FIV+', 'FLUTD', 
           'Prescription diet- adopter obtaining themselves', 
           'Chronic constipation', 'FeLV Regressive', 
           'Musculoskeletal abnormality', 'Care for Life Agreement', 
           'Heart disease / HCM', 'Dental disease- moderate', 
           'Prescription diet- being provided by shelter', 
           'Dental disease- mild', 'FeLV+', 'IBD', 'Skin allergy',  
           'Heart murmur with proBNP', 'Dental disease- stomatitis', 
           'Hyperthyroid', 'FeLV Discordant', 'Chronic kidney disease', 
           'Chronic Condition Support', 'Kitten Rabies Disclaimer', 
           'URI- chronic', 'Dental disease- severe', 'Epilepsy/seizure disorder', 
           'Cerebellar hypoplasia', 'Asthma']

    behavioral = ['Bonded', 'Undersocialized', 'Overstimulation', 'Housesoiling', 
                'Barn Cat', 'Media', 'Best without other cats', 'Best without kids', 
                'High energy', 'Best without dogs']

    both = ["Declawed"]

    if elem in medical:
        return "Medical"
    elif elem in behavioral:
        return "Behavioral"
    elif elem in both:
        return "Both"

In [39]:
hardtoplace['Type'] = hardtoplace["Attributes"].apply(attribute_category)

In [40]:
# Noting if a cat has one of the "hard to place" issues or not
cat_data["Hard to Place"] = cat_data["Animal ID"].isin(
    hardtoplace["Animal ID"].unique().tolist()
    )

In [41]:
# personal preference on data format
cat_data["Hard to Place"] = cat_data["Hard to Place"].replace(
    {True:"Yes", False:"No"}
    )

In [42]:
adopter = pd.read_excel(folder + "adopter.xlsx")

In [43]:
adopter.columns

Index(['Date Of Adoption', 'Animal ID', 'Animal Name', 'Species',
       'Primary Breed', 'Sex', 'Altered', 'Est. Birthdate', 'Microchip Number',
       'By (User)', 'Outcome Subtype', 'Adopter ID', 'Adopter Name',
       'Primary Phone', 'Primary Email', 'Street Address', 'Unit', 'City',
       'State', 'Zip'],
      dtype='object')

I needed to drop identifying information from the adoption data

In [44]:
adopter = adopter.drop(columns=["Adopter Name", 'Primary Phone', 
                                'Primary Email', 'Street Address', 'Unit'])

In [45]:
savepath = "/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/clean/"
cat_data.to_csv(savepath + "catdata.csv")
hardtoplace.to_csv(savepath + "hardtoplace.csv")
adopter.to_csv(savepath + "adopter.csv")